In [ ]:
from google_apis import create_service

client_secret_file = 'client_secret_2.json'
API_SERVICE_NAME = 'gmail'
API_VERSION = 'v1'
SCOPES = ['https://mail.google.com/']

service = create_service(client_secret_file, API_SERVICE_NAME, API_VERSION, SCOPES)

gmail API v1 service created successfully


In [7]:
dir(service)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_basic_methods',
 '_add_nested_resources',
 '_add_next_methods',
 '_baseUrl',
 '_credentials_validated',
 '_developerKey',
 '_dynamic_attrs',
 '_http',
 '_model',
 '_requestBuilder',
 '_resourceDesc',
 '_rootDesc',
 '_schema',
 '_set_dynamic_attr',
 '_set_service_methods',
 '_universe_domain',
 '_validate_credentials',
 'close',
 'new_batch_http_request',
 'users']

In [8]:
import marimo as mo
from gmail_api import init_gmail_service, get_email_messages, get_email_message_details  
client_file = "client_secret_2.json"
service = init_gmail_service(client_file)

gmail API v1 service created successfully


In [9]:
messages = get_email_messages(service)
print(f"Total messages fetched: {len(messages)}")

Total messages fetched: 444


In [10]:
import re
from datetime import datetime
import openpyxl

# Create or load an Excel workbook
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Transactions"

# Add headers if not already present
if ws.max_row == 1:
    ws.append(["Sender", "Receiver", "Amount", "In/Out"])

now = datetime.now()

# Get current year
year = now.strftime("%Y")

# Get current month as an integer (01 to 12)
monthData = now.strftime("%m")
monthList = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]

# Convert monthData to integer and use it to index monthList (0-based index)
monthIndex = int(monthData) - 1  # Adjust for 0-based index
month = monthList[monthIndex]
day = now.strftime("%d")
time = now.strftime("%a")

# Format date as "Day, dd Month yyyy"
curr_date = f"{time}, {day} {month} {year}"

# Variables to track total amounts
total_in = 0.0
total_out = 0.0

# Function to process payment message
def process_payment_message(subject):
    # Check if the subject is of a payment type
    send_pattern = r"You sent \$(\d+\.?\d*) to ([A-Za-z ]+)"
    receive_pattern = r"([A-Za-z ]+) sent you \$(\d+\.?\d*)"

    send_match = re.match(send_pattern, subject)
    receive_match = re.match(receive_pattern, subject)

    if send_match:
        amount = float(send_match.group(1))  # Convert to float for calculations
        receiver = send_match.group(2)
        sender = "You"
        in_out = "Out"
        return sender, receiver, amount, in_out

    elif receive_match:
        sender = receive_match.group(1)
        amount = float(receive_match.group(2))  # Convert to float for calculations
        receiver = "You"
        in_out = "In"
        return sender, receiver, amount, in_out

    return None

for msg in messages:
    details = get_email_message_details(service, msg['id'])
    email_date_str = details["date"]

    # Replace 'GMT' with '+0000' for proper parsing
    email_date_str = email_date_str.replace(" GMT", " +0000")
    
    try:
        # Parse the email date
        email_date_obj = datetime.strptime(email_date_str, "%a, %d %b %Y %H:%M:%S %z")
        email_date = email_date_obj.strftime("%a, %d %b %Y")
        
        if email_date == curr_date:
            subject = details['subject']
            payment_info = process_payment_message(subject)

            if payment_info:
                sender, receiver, amount, in_out = payment_info

                # Append transaction to the Excel sheet
                ws.append([sender, receiver, amount, in_out])

                # Update totals
                if in_out == "In":
                    total_in += amount
                elif in_out == "Out":
                    total_out += amount

                print(f"Recorded: {sender} -> {receiver} | Amount: ${amount} | {in_out}")

    except ValueError as e:
        print(f"Error parsing date: {email_date_str}")
        print(f"Exception: {e}")

# Calculate profit/loss
profit_loss = total_in - total_out

# Write summary section
summary_row = ws.max_row + 2  # Leave a gap after transactions
ws[f"A{summary_row}"] = "Summary"
ws[f"A{summary_row + 1}"] = "Total In"
ws[f"B{summary_row + 1}"] = total_in

ws[f"A{summary_row + 2}"] = "Total Out"
ws[f"B{summary_row + 2}"] = total_out

ws[f"A{summary_row + 3}"] = "Profit/Loss"
ws[f"B{summary_row + 3}"] = profit_loss

# Save the workbook
wb.save("transactions.xlsx")

print(f"Summary Recorded: Total In: ${total_in}, Total Out: ${total_out}, Profit/Loss: ${profit_loss}")


Recorded: Rasheed Cosby -> You | Amount: $5.0 | In
Recorded: Dave Shabeeb -> You | Amount: $10.0 | In
Recorded: Queen Dorsey -> You | Amount: $10.0 | In
Recorded: Queen Dorsey -> You | Amount: $10.0 | In
Recorded: John David Janosh -> You | Amount: $15.0 | In
Recorded: Queen Dorsey -> You | Amount: $5.0 | In
Recorded: John David Janosh -> You | Amount: $5.0 | In
Recorded: John David Janosh -> You | Amount: $15.0 | In
Recorded: mary -> You | Amount: $5.66 | In
Recorded: mary -> You | Amount: $6.0 | In
Recorded: You -> Bob Garcia | Amount: $50.0 | Out
Recorded: Rasheed Cosby -> You | Amount: $5.0 | In
Summary Recorded: Total In: $91.66, Total Out: $50.0, Profit/Loss: $41.66
